# **Bitcoin price forecasting - Random Forest Regressor**
### Big Data Computing final project - A.Y. 2022 - 2023
Prof. Gabriele Tolomei

MSc in Computer Science

La Sapienza, University of Rome

### Author
Corsi Danilo - corsi.1742375@studenti.uniroma1.it



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Dependencies, Libraries and Tools

In [3]:
JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64"
SLOW_OPERATION = False

In [4]:
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
#Install some useful dependencies
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from itertools import cycle

import plotly.express as px

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import gc

import pandas as pd
import numpy as np
import json

import matplotlib.pyplot as plt
import seaborn as sns

# !pip install -U -q PyDrive # To use files that are stored in Google Drive directly (e.g., without downloading them from an external URL)
# !apt install openjdk-8-jdk-headless -qq
# import os
# os.environ["JAVA_HOME"] = JAVA_HOME

In [6]:
# Install Spark and related dependencies
!pip install pyspark

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

from pyspark.ml.regression import LinearRegression, RandomForestRegressor, GBTRegressor
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation
from pyspark.ml import Pipeline

# Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '4G').\
                set('spark.driver.memory', '45G').\
                set('spark.driver.maxResultSize', '10G').\
                set("spark.kryoserializer.buffer.max", "1G").\
                setAppName("BitcoinPriceForecasting").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285397 sha256=6970c5e08c36b16dee6c3e7d5fe0cd7161b59fdc77972b3d647e1493896f0eae
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


# Link to Google Drive

In [7]:
# Define GDrive paths
GDRIVE_DIR = "/content/drive"

GDRIVE_DATASET_RAW_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/datasets/raw"
GDRIVE_DATASET_TEMP_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/datasets/temp"
GDRIVE_DATASET_OUTPUT_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/datasets/output"

GDRIVE_DATASET_NAME = "bitcoin_blockchain_data_30min"
GDRIVE_DATASET_NAME_TRAIN = GDRIVE_DATASET_NAME + "_train"
GDRIVE_DATASET_NAME_VALID = GDRIVE_DATASET_NAME + "_valid"

# GDRIVE_DATASET_NAME_EXT = "/" + GDRIVE_DATASET_NAME + ".parquet"
GDRIVE_DATASET_NAME_EXT_TRAIN  = "/" + GDRIVE_DATASET_NAME_TRAIN + ".parquet"
GDRIVE_DATASET_NAME_EXT_VALID = "/" + GDRIVE_DATASET_NAME_VALID + ".parquet"

# GDRIVE_DATASET = GDRIVE_DATASET_RAW_DIR + GDRIVE_DATASET_NAME_EXT
GDRIVE_DATASET_TRAIN = GDRIVE_DATASET_OUTPUT_DIR + GDRIVE_DATASET_NAME_EXT_TRAIN
GDRIVE_DATASET_VALID = GDRIVE_DATASET_OUTPUT_DIR + GDRIVE_DATASET_NAME_EXT_VALID

In [8]:
# Point Colaboratory to our Google Drive
from google.colab import drive

drive.mount(GDRIVE_DIR, force_remount=True)

Mounted at /content/drive


In [9]:
# Load datasets into pyspark dataframe objects
train_df = spark.read.load(GDRIVE_DATASET_TRAIN,
                         format="parquet",
                         sep=",",
                         inferSchema="true",
                         header="true"
                    )

valid_df = spark.read.load(GDRIVE_DATASET_VALID,
                         format="parquet",
                         sep=",",
                         inferSchema="true",
                         header="true"
                    )

# Cache it
train_df.cache()
valid_df.cache()

DataFrame[market-price: double, market-cap: double, total-bitcoins: double, trade-volume: double, blocks-size: double, avg-block-size: double, n-transactions-total: double, n-transactions-per-block: double, hash-rate: double, difficulty: double, miners-revenue: double, transaction-fees-usd: double, n-unique-addresses: double, n-transactions: double, estimated-transaction-volume-usd: double, timestamp: timestamp_ntz, index: int]

# Import my utilities ❗

In [39]:
import sys
GDRIVE_UTILITIES_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/utilities"
sys.path.append(GDRIVE_UTILITIES_DIR)

import shutil
shutil.rmtree(GDRIVE_UTILITIES_DIR + '/__pycache__')

import utilities

import importlib
importlib.reload(utilities)

<module 'utilities' from '/content/drive/MyDrive/BDC/project/utilities/utilities.py'>

# Training simple model ❗

In [12]:
# Retrieve all / cor_matrix / gb features
GDRIVE_FEATURES_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/features"

GDRIVE_COR_MATRIX_FEATURES_NAME = "cor_matrix_features"

GDRIVE_COR_MATRIX_FEATURES_NAME_EXT = "/" + GDRIVE_COR_MATRIX_FEATURES_NAME + ".json"

GDRIVE_COR_MATRIX_FEATURES = GDRIVE_FEATURES_DIR + GDRIVE_COR_MATRIX_FEATURES_NAME_EXT

In [13]:
all_features = train_df.columns[1:-2]

cor_matrix_features = spark.read.json(GDRIVE_COR_MATRIX_FEATURES).columns

# Set the depended variable
dep_var = 'market-price'

In [40]:
# Valid performances with all the features
utilities.train_valid_simple_model(train_df, valid_df, 'RandomForestRegressor', all_features, 'features', dep_var)

Output hidden; open in https://colab.research.google.com to view.

In [41]:
# Valid performances with the corr matrix features
utilities.train_valid_simple_model(train_df, valid_df, 'RandomForestRegressor', cor_matrix_features, 'features', dep_var)

Output hidden; open in https://colab.research.google.com to view.

# Hyperparameter tuning ❗

In [42]:
combined_df = train_df.union(valid_df)

# Release Cache
train_df.unpersist()
valid_df.unpersist()

DataFrame[market-price: double, market-cap: double, total-bitcoins: double, trade-volume: double, blocks-size: double, avg-block-size: double, n-transactions-total: double, n-transactions-per-block: double, hash-rate: double, difficulty: double, miners-revenue: double, transaction-fees-usd: double, n-unique-addresses: double, n-transactions: double, estimated-transaction-volume-usd: double, timestamp: timestamp_ntz, index: int]

In [43]:
combined_df = utilities.select_features(combined_df, cor_matrix_features, dep_var)

In [44]:
# Split proportion list
proportion_lst = [0.6, 0.7, 0.8, 0.9]

In [45]:
# RandomForest params
# params = {
#     'numTrees' : [3, 5, 10, 20, 30],# Number of trees to train, >=1, default:20
#     'maxDepth' : [3, 5, 10] # Maximum depth of the tree, <=30, default:5
# }

params = {
    'numTrees' : [3, 6, 9, 12, 15],# Number of trees to train, >=1, default:20
    'maxDepth' : [3, 5, 10] # Maximum depth of the tree, <=30, default:5
}

In [46]:
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.ml.regression import LinearRegression, RandomForestRegressor, GBTRegressor
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation
from pyspark.ml import Pipeline

# Apache Spark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler,StandardScaler
from pyspark.ml.regression import LinearRegression, GeneralizedLinearRegression, DecisionTreeRegressor, RandomForestRegressor, GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator

# Python
import numpy as np
import pandas as pd
from itertools import product
import time

# Graph packages
# https://plotly.com/python/getting-started/#jupyterlab-support
# https://plotly.com/python/time-series/
import plotly.express as px

# Scikit-learn
from sklearn.metrics import mean_absolute_percentage_error

#Install some useful dependencies
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import cycle

import plotly.express as px

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import gc

import pandas as pd
import numpy as np
import json

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
results = utilities.autoTuning(combined_df, proportion_lst, "RandomForestRegressor", "features", dep_var, params)
results

# Time Series Cross Validation ❗

In [ ]:
## Cross Validation Parameter
# Multiple Splits Time Series Cross Validation
mul_cv = {'cv_type':'mulTs',
          'kSplits': 5}

# Blocked Time Series Cross Validation
blk_cv = {'cv_type':'blkTs',
          'kSplits': 10}

In [ ]:
# RandomForest params
params = {
    'numTrees' : [6],# Number of trees to train, >=1, default:20
    'maxDepth' : [10] # Maximum depth of the tree, <=30, default:5
}

In [ ]:
results_mul_cv, trained_models_mul_cv = utilities.tsCrossValidation(combined_df, "RandomForestRegressor", "features", dep_var, params, mul_cv)
results_mul_cv

In [ ]:
results_blk_cv, trained_models_blk_cv = utilities.tsCrossValidation(combined_df, "RandomForestRegressor", "features", dep_var, params, blk_cv)
results_blk_cv

# Model Comparison Table

In [ ]:
# Define what model_info and evaluators in the Model Comparison Table
model_info = ['Model','CV_type','Parameters']
evaluator_lst = ['RMSE','MAPE','MAE','Variance','R2','Adjusted_R2','Time']

# The the Cross Validation results would like to compare
comparison_lst = [results_mul_cv, results_blk_cv]

In [ ]:
# Show the Comparison Table
pd.concat([utilities.modelComparison(cv_result ,model_info,evaluator_lst) for cv_result in comparison_lst])

In [ ]:
GDRIVE_MODEL_NAME = "RandomForestRegressor"
GDRIVE_MODELS_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/models"
GDRIVE_MODEL_NAME_EXT = GDRIVE_MODELS_DIR + "/" + GDRIVE_MODEL_NAME

In [ ]:
import joblib

# Save the best models
for i, model in enumerate(results_blk_cv):
    joblib.dump(model, f"{GDRIVE_MODEL_NAME_EXT}/model_{i}.joblib")